In [49]:
import pandas as pd
import gc
import numpy as np

df = pd.read_csv("../datasets/sell-in.txt.gz", sep="\t")
df = df.groupby(by=["periodo","product_id"]).agg({"tn":"sum"}).reset_index()
df["periodo"] = pd.to_datetime(df["periodo"], format="%Y%m")
df_pivot = df.pivot(index="periodo", columns="product_id", values="tn").reset_index()

magicos = [ "periodo",20002, 20003, 20006, 20010, 20011, 20018, 20019, 20021,
   20026, 20028, 20035, 20039, 20042, 20044, 20045, 20046,  20049,
   20051, 20052, 20053, 20055, 20008, 20001, 20017, 20086, 20180,
   20193, 20320, 20532, 20612, 20637, 20807, 20838
 ]
X_train = df_pivot[magicos].query("periodo >= '2018-01-01' & periodo <= '2018-12-31' ")
X_train = X_train.T.iloc[1:]
X_train.columns = [f"t-{11-k}" for k in range(12)]


X_kgl = df_pivot.query("periodo >= '2019-01-01' & periodo <= '2019-12-31'")
X_kgl = X_kgl.T.iloc[1:]
X_kgl.columns = [f"t-{11-k}" for k in range(12)]
promedio = X_kgl.mean(axis=1)
promedio = promedio.fillna(0)


y = df_pivot[magicos].query("periodo == '2019-02-01'").T.iloc[1:]
y.columns = ["target"]


C:\Users\carre\AppData\Local\Temp\ipykernel_1940\2930753992.py:24: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  promedio = promedio.fillna(0)
C:\Users\carre\AppData\Local\Temp\ipykernel_1940\2930753992.py:27: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  y = df_pivot[magicos].query("periodo == '2019-02-01'").T.iloc[1:]


In [50]:
y

,target
product_id,
20002,1043.01349
20003,758.32657
20006,479.99914
20010,337.76009
20011,431.62938
20018,265.84135
20019,323.66178
20021,219.88515
20026,263.67957


In [51]:
promedio

product_id
20001    1454.732720
20002    1175.437142
20003     784.976407
20004     627.215328
20005     668.270104
            ...     
21295       0.000000
21296       0.000000
21297       0.000000
21298       0.000000
21299       0.000000
Length: 1233, dtype: float64

In [52]:
prod_menos12 = X_kgl.index[X_kgl.isna().sum(axis=1)> 0]
X_kgl = X_kgl[~X_kgl.index.isin(prod_menos12)]
promedio_menos12 = promedio[prod_menos12]

HIPERPARAMETROS

In [53]:
#predict 
productos_ok = pd.read_csv("https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/product_id_apredecir201912.txt", sep="\t")


In [54]:
from sklearn.linear_model import LinearRegression

reg_model = LinearRegression()
reg_model.fit(X_train, y)

LinearRegression()

In [55]:
reg_model.intercept_

array([0.44146718])

In [56]:
reg_model.coef_

array([[ 0.07367052,  0.11921107,  0.10380433,  0.14283912,  0.04393265,
         0.15193647, -0.00777452, -0.16187541, -0.0600306 ,  0.17820788,
         0.23655828, -0.00133878]])

In [57]:
X_kgl

,t-11,t-10,t-9,t-8,t-7,t-6,t-5,t-4,t-3,t-2,t-1,t-0
product_id,,,,,,,,,,,,
20001,1275.77351,1259.09363,1470.65653,1647.63848,1629.78233,1109.93769,1678.99318,1261.34529,1660.00561,1561.50552,1397.37231,1504.68856
20002,1266.78751,1043.01349,1083.62552,1287.62346,1034.98927,928.36431,1066.44999,813.78215,1090.18771,1979.53635,1423.57739,1087.30855
20003,964.76919,758.32657,638.0401,565.33774,590.12515,662.38654,715.20314,635.59563,967.77116,1081.36645,948.29393,892.50129
20004,511.33713,441.70332,619.77084,466.70901,603.31081,667.19411,521.71519,482.13372,786.1714,1064.69633,723.94206,637.90002
20005,363.58438,409.8995,488.21387,624.9988,897.26297,876.39696,745.74978,536.668,879.52808,996.78275,606.91173,593.24443
...,...,...,...,...,...,...,...,...,...,...,...,...
21257,0.01421,0.00591,0.01299,0.02469,0.09221,0.0118,0.08213,0.00524,0.01596,0.05176,0.00677,0.01158
21259,0.04234,0.0508,0.06777,0.02117,0.01976,0.00988,0.04657,0.03106,0.00564,0.01975,0.02965,0.01412
21262,0.0268,0.05786,0.06353,0.01412,0.0113,0.0127,0.03953,0.02258,0.01552,0.01693,0.02682,0.01834


In [58]:
len(X_kgl.index)

736

In [59]:
pred = pd.DataFrame({"product_id": X_kgl.index, "tn": reg_model.predict(X_kgl).flatten()})
nuevas_filas = []
for prod in productos_ok["product_id"]:
    if prod not in pred["product_id"].values:
        nuevas_filas.append({"product_id": prod, "tn": promedio[prod]})

pred = pd.concat([pred, pd.DataFrame(nuevas_filas)], ignore_index=True)
pred = pred[pred["product_id"].isin(productos_ok["product_id"])]
pred

,product_id,tn
0,20001,1162.707525
1,20002,1183.640604
2,20003,684.763931
3,20004,580.484961
4,20005,563.560780
...,...,...
861,21252,0.178011
862,21265,0.089541
863,21266,0.094659
864,21267,0.092835


In [60]:
pred.to_csv("../results/reg_lineal5.csv", index=False, sep=",")